In [2]:
import pandas as pd
import ast
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel



In [3]:
steam = pd.read_csv('steam.csv')


In [3]:
# Observamos los datos
steam.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28728 entries, 0 to 28727
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    28728 non-null  int64 
 1   id            28728 non-null  int64 
 2   genres        28728 non-null  object
 3   app_name      28728 non-null  object
 4   release_date  28728 non-null  object
 5   year          28728 non-null  int64 
dtypes: int64(3), object(3)
memory usage: 1.3+ MB


In [4]:
games = steam[['id','app_name','genres']].copy()

In [58]:
games.head()

,id,app_name,genres
0,761140,Lost Summoner Kitty,"['Action', 'Casual', 'Indie', 'Simulation', 'S..."
1,643980,Ironbound,"['Free to Play', 'Indie', 'RPG', 'Strategy']"
2,670290,Real Pool 3D - Poolians,"['Casual', 'Free to Play', 'Indie', 'Simulatio..."
3,767400,弹炸人2222,"['Action', 'Adventure', 'Casual']"
4,772540,Battle Royale Trainer,"['Action', 'Adventure', 'Simulation']"


In [5]:
games.shape

(28728, 3)

In [6]:
games = games.drop_duplicates()

In [8]:
games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 28727 entries, 0 to 28727
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        28727 non-null  int64 
 1   app_name  28727 non-null  object
 2   genres    28727 non-null  object
dtypes: int64(1), object(2)
memory usage: 897.7+ KB


In [14]:
muestra_games = games.sample(n=5000) 



In [16]:
muestra_games = muestra_games.reset_index(drop=True)
muestra_games.head()


,id,app_name,genres
0,275830,7 Wonders: Magical Mystery Tour,"['Casual', 'Strategy']"
1,329150,Vanguard Princess Lilith,"['Action', 'Casual', 'Indie']"
2,353940,Aces of the Luftwaffe,['Action']
3,590630,Fantasy Grounds - A Wedding at Axebridge (RMC),"['Indie', 'RPG', 'Strategy']"
4,3520,Mystery P.I.™ - The Vegas Heist,['Casual']


In [24]:
muestra_games.to_csv('data_modelo.csv')

In [21]:
def recomendacion_juego(product_id):
    # Calcula la matriz TF-IDF para los géneros de juegos
    tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=500)
    tfidf_matrix = tfidf_vectorizer.fit_transform(muestra_games['genres'])
    
    # Calcula la similitud de coseno entre los juegos
    cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
    
    # Encuentra el índice del juego con el ID dado
    index = muestra_games[muestra_games['id'] == product_id].index[0]
    
    # Calcula las puntuaciones de similitud de coseno para el juego en cuestión
    sim_scores = list(enumerate(cosine_sim[index]))
    
    # Ordena los juegos en función de las puntuaciones de similitud
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Obtiene los índices de los juegos más similares (excluyendo el juego de entrada)
    sim_scores = sim_scores[1:6]  # Cambia esto si quieres más o menos recomendaciones
    
    # Obtiene los títulos de los juegos recomendados
    game_indices = [i[0] for i in sim_scores]
    recommended_game_titles = muestra_games.iloc[game_indices]['app_name']
    
    return recommended_game_titles


In [22]:
recomendacion_juego(275830)

231                                    Zombie Quarantine
366                     Garden Rescue: Christmas Edition
382                                           Battleship
544    Fritz for Fun 13: Chessbase Power Play Tutoria...
786                         Crazy Machines: Golden Gears
Name: app_name, dtype: object